In [ ]:
! pip install deepface

In [3]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from deepface import DeepFace

In [3]:
def extract_features(img_path):
  embedding_objs = DeepFace.represent(img_path, model_name='ArcFace', enforce_detection=False)
  return embedding_objs[0]['embedding']

In [4]:
from tqdm import tqdm

In [13]:
root = '/content/drive/MyDrive/datasets/PersianFace'
data = []
dirs = os.listdir(root)
for dir in tqdm(dirs):
  for img in os.listdir(os.path.join(root, dir)):
     features = extract_features(os.path.join(root, dir, img))
     features.insert(0, dir)
     data.append(features)

100%|██████████| 30/30 [08:59<00:00, 18.00s/it]


In [14]:
column_names = ['person name']
for i in range(512):
  column_names.append(f'f{i+1}')

df_org = pd.DataFrame(data, columns=column_names)
df_org.to_csv('face_features.csv', index=False)
df_org.head()

,person name,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f503,f504,f505,f506,f507,f508,f509,f510,f511,f512
0,Ali_Daei,0.094716,0.296553,-0.058460,-0.036193,-0.253264,0.031777,-0.069927,-0.264900,-0.177623,...,-0.063134,0.185177,-0.230704,-0.073284,-0.147854,-0.160688,-0.237804,0.520278,-0.162291,-0.430183
1,Ali_Daei,0.025459,-0.086801,0.116795,-0.121392,-0.054967,-0.123564,-0.008367,0.056431,0.180552,...,-0.257911,-0.141294,-0.049147,-0.053520,0.042037,0.043071,-0.093048,-0.031122,0.175395,-0.123276
2,Ali_Daei,-0.022762,-0.071330,-0.200252,0.151588,-0.067989,0.199873,0.082630,0.016104,0.031681,...,-0.164835,0.011315,-0.150421,0.077362,0.064151,0.145640,-0.183997,0.192758,0.046455,-0.119670
3,Ali_Daei,0.010319,-0.179178,-0.176310,-0.234156,-0.190156,-0.006309,0.057098,-0.008688,0.158632,...,-0.396135,-0.015876,-0.268975,0.174640,0.095728,-0.031581,-0.362334,0.291133,-0.035755,-0.402195
4,Ali_Daei,0.068914,0.072270,0.057946,-0.058213,-0.157949,0.004092,0.022809,0.036618,-0.009681,...,-0.083237,-0.078793,-0.028144,0.087475,-0.030747,0.252180,-0.075436,0.244707,0.259619,-0.074900


In [15]:
df = df_org.replace(dirs, [i for i in range(len(dirs))])
df = df.sample(frac=1)
df.head()

,person name,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f503,f504,f505,f506,f507,f508,f509,f510,f511,f512
1300,22,-0.335922,0.603518,-0.164973,-0.676987,0.049577,0.476392,-0.241575,0.001663,-0.089141,...,-1.580610,0.272533,0.518322,-0.859409,0.293458,-0.387514,0.007564,0.175383,-0.420130,-1.710191
1609,27,-0.247172,0.516746,-0.174688,-0.406971,0.058111,0.357695,-0.145446,-0.107280,-0.007230,...,-0.997778,0.125681,0.372099,-0.718709,0.337414,-0.331906,-0.026775,0.057123,-0.500228,-1.227442
770,13,-0.027702,0.121022,0.097605,-0.031033,-0.045835,0.010965,-0.022229,0.007106,0.034723,...,-0.125042,-0.141486,-0.098749,-0.050885,0.219217,-0.061504,-0.001214,0.028112,0.142287,-0.060921
1483,25,-0.017516,-0.186967,0.233863,0.146296,-0.109934,-0.089072,-0.012778,0.201739,0.199729,...,-0.083901,0.216744,-0.365898,-0.035139,0.205361,0.152609,0.015344,-0.113762,0.069843,0.210797
1654,28,0.046105,0.273901,-0.036782,0.130469,-0.051134,-0.054592,-0.011054,-0.017552,0.041521,...,0.115712,-0.148451,-0.155175,-0.160392,-0.020842,-0.049778,-0.101694,0.455205,0.011214,0.074979


In [16]:
cut = round(len(df) * .8)

x_train = df.iloc[ :cut , 1:]
y_train = df['person name'][ :cut]
x_test = df.iloc[cut: , 1:]
y_test = df['person name'][cut: ]

In [17]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, 'tanh', input_shape=(512,)),
    tf.keras.layers.Dense(256, 'relu'),
    tf.keras.layers.Dense(64, 'relu'),
    tf.keras.layers.Dense(len(dirs), 'softmax')
])
model.compile(tf.keras.optimizers.Adamax(.0001), 'sparse_categorical_crossentropy', metrics=['accuracy'])
output = model.fit(x_train, y_train, epochs=60, validation_data=(x_test, y_test))

Epoch 1/60
44/44 [==============================] - 2s 9ms/step - loss: 3.3046 - accuracy: 0.1738 - val_loss: 3.1867 - val_accuracy: 0.3014
Epoch 2/60
44/44 [==============================] - 0s 4ms/step - loss: 3.0760 - accuracy: 0.4012 - val_loss: 3.0014 - val_accuracy: 0.4145
Epoch 3/60
44/44 [==============================] - 0s 4ms/step - loss: 2.8581 - accuracy: 0.4823 - val_loss: 2.8051 - val_accuracy: 0.4638
Epoch 4/60
44/44 [==============================] - 0s 5ms/step - loss: 2.6346 - accuracy: 0.5402 - val_loss: 2.6000 - val_accuracy: 0.5043
Epoch 5/60
44/44 [==============================] - 0s 4ms/step - loss: 2.4085 - accuracy: 0.5735 - val_loss: 2.4057 - val_accuracy: 0.5391
Epoch 6/60
44/44 [==============================] - 0s 4ms/step - loss: 2.1984 - accuracy: 0.6235 - val_loss: 2.2244 - val_accuracy: 0.5739
Epoch 7/60
44/44 [==============================] - 0s 5ms/step - loss: 2.0077 - accuracy: 0.6430 - val_loss: 2.0635 - val_accuracy: 0.6000
Epoch 8/60
44/44 [==

In [12]:
model.save('weights/model.h5')

In [ ]:
feature_vector1 = extract_features('assets/javad razavian 1.jpg')
feature_vector2 = extract_features('assets/javad razavian 2.jpg')
feature_vector3 = extract_features('assets/mehran modiri.jpg')
print(dirs[np.argmax(model.predict([feature_vector1]))])
print(dirs[np.argmax(model.predict([feature_vector2]))])
print(dirs[np.argmax(model.predict([feature_vector3]))])

1/1 [==============================] - 0s 63ms/step
Bahram_Radan
1/1 [==============================] - 0s 55ms/step
Javad_Razavian
1/1 [==============================] - 0s 43ms/step
Mehran_Modiri
